In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import scipy.stats as stats
import astropy.stats as astats
import numpy.random as random
import pandas as pd

# Fitting for the Hubble constant with supernovae

In this notebook, we are going to learn to use linear regression to fit the hubble constant, H0, using Hubble's law:

$v \approx cz = H_0D$

We are going to work with the Hicken et al. ("Constitution") supernova dataset.  First, we read in the file into a pandas dataframe named `data`:

In [ ]:
# CHANGE THE BELOW LINE TO POINT TO THE DIRECTORY CONTAINING SNDATA.TXT
path = ''  

# the pandas way: the file is in "fixed-width format" so we use read_fwf
data=pd.read_fwf(path+'sndata.txt')

In the cell below, let's open up the data and take a look at the format

In [ ]:
data

As you can see, the data is stored with cz, a variable we would like to use, but the distances are in terms of the distance modulus, mu, which is related to the distance d in parsecs as follows:

$log_{10}(d) = 1+\frac{\mu}{5}$

Solving for d:

$d = 10^{1+\frac{\mu}{5}}$

The Hubble constant is often measured in units of km/s/Mpc, so we'll be converting everything to fit our slope in those units.

Let's read in the data and plot.

In [ ]:
cz=data['cz'] #already in km/s
mu=data['mu']
sigma_mu=data['sigma_mu']
d = 10**(1+mu/5) * 1e-6 #convert to Mpc

plt.plot(cz,d, 'b.')
plt.xlabel('cz (km/s)')
plt.ylabel(r'Distance (Mpc)')
plt.ylim(0,500)
plt.show()

plt.errorbar(cz,mu,yerr=sigma_mu,fmt='b.')
plt.xlabel('cz (km/s)')
plt.ylabel(r'$\mu$')
plt.show()

Since our measurements are made on $\mu$ and therefore our errors are in $\mu$, we're going to fit in logarithmic space. This is extremely common in astronomy, as our measurements often span orders of magnitude in brightness/distance.

$d = \frac{cz}{H_0} x 10^6$ parsecs

$log_{10}(d) = 1+\frac{\mu}{5} = log_{10}(\frac{cz}{H_0})+ 6$

$\mu = 5 log_{10}(cz) + 5[5-log_{10}(H_0)]$

So, if we fit a line to this, the intercept, b, will be:

$b = 5[5-log_{10}(H_0)]$

And $H_0$ will be:

$H_0 = 10^5 10^{-0.2*b} \mathrm{\frac{km/s}{Mpc}}$

<b>In the cell below, plot $\mu$ as a function of $\mathrm{log_{10}(cz)}$. Name the variable for the log of cz "logv":</b>

Now, let's fit a line to this data and get $H_0$ from the intercept. We can use the scipy.stats linregress function to do this. If this worked properly, our slope should be very close to 5, so we can print that as a check.

In [ ]:
#define a function to convert from the intercept to H0 to make things easier
def int_to_H0(b):
    return(10**(-0.2*b) * 10**5)

slope,intercept,r,p,s = stats.linregress(logv,mu)

plt.plot(logv,mu,'b.')
plt.ylim(33,39)
plt.xlabel('log v')
plt.ylabel(r'$\mu$')
plt.plot(logv,logv*slope+intercept,'r-')
print('slope: ',slope,' +/- ',s)
print('intercept: ',intercept)
print('H0: ',int_to_H0(intercept))

mu_fit = slope*logv+intercept

We got $H_0$ as very close to 70, which is very close to the accepted value! However, we didn't incorporate our errors into our measurement at all. In order to do that, let's do an inverse variance weighted least-squares instead. We can use the numpy polyfit routine to do this. 

In [ ]:
weight = 1./sigma_mu**2

#call polyfit with our inverese variance weights and fit a 1st order (linear) polynomial
coeffs, covar = np.polyfit(logv,mu,1,w=weight,cov=True)
slope = coeffs[0]
intercept = coeffs[1]
intercept_err = np.sqrt(covar[1,1])
s=np.sqrt(covar[0,0])

print('slope: ',slope,' +/- ',s)
print('intercept: ',intercept)

#to get the error in H0 from this method, take the average of H0 from the intercept +1 sigma and -1 sigma errors
h0err = (int_to_H0(intercept-intercept_err)-int_to_H0(intercept+intercept_err))/2
print('H0: ',int_to_H0(intercept),'+/-',h0err)

plt.errorbar(logv,mu,yerr=sigma_mu,fmt='b.')
plt.ylim(33,39)
plt.xlabel('log v')
plt.ylabel(r'$\mu$')
plt.plot(logv,logv*slope+intercept,'r-')
plt.show()

mu_fit_err = slope*logv+intercept

To evaluate how good a fit is, we use a parameter called $\chi^2$:

$\chi^2 = \Sigma_i \frac{(y_i-f(x_i, \beta))^2}{\sigma_i^2}$

$f(x_i, \beta)$ is our model in this case, which is a function of x (in our case, log(v)) and some parameters $\beta$, in our case the slope and the intercept.

The goal of regression is to minimize $\chi^2$; the model which gives the lowest value is the best fit. In the above cases, the weighted and unweighted methods are the same, the unweighted just sets sigma=1 for all points so that they have equal weight in the sum.

In the below cell, calculate $\chi^2$ for the inverse variance weighted model:

In [ ]:
chi2_linear = np.sum(?)
print('chi2: ', chi2_linear)

In the above case, we assumed that Hubble's law had a linear form and found a good fit to the data, but that doesn't have to be the case. What if instead, there's both linear and quadratic dependence on v? Let's try doing that fit. Instead of evaluating by hand, we can use the numpy polyval function. 

In [ ]:
coeffs, covar = np.polyfit(logv,mu,2,w=weight,cov=True)
print('quadratic coefficients', coeffs)
# note the order of the coefficients: the quadratic term is first, constant last

quad_fit = np.polyval(coeffs,logv)

poly_quad = np.poly1d(coeffs)
quad_fit2  = poly_quad(logv)

#make a plot with the linear and quadtratic lines plotted along with the points
???

You should see that both of these fits perform very similarly, but can we assess which model actually fits better? Find $\chi^2$ for the quadratic fit and see which fit minimizes $\chi^2$ more.

In [ ]:
#find the quadratic chi^2 and print it, along with the linear one
???

print(chi2_linear, chi2_quad)

You should find that the quadratic model outperforms the linear one. That <b>has to</b> be the case, because we've just taken the linear model and added more parameters. Adding more parameters to an existing model will <b>always</b> improve $\chi^2$, or leave it basically unchanged. But in that case, why not just fit a 1000th order polynomial every time?

There are a few answers here. The first is that we want our models to be physically motivated so that they can extrapolate. Let's take a look in the below cell at how our two models look when we go outside the range of our data, and let's also add third, fourth, and 50th order fits:

In [ ]:
coeffs_lin, covar_lin = np.polyfit(logv,mu,1,w=weight,cov=True)
coeffs_quad, covar_quad = ?
coeffs_cub, covar_cub = ?
coeffs_quar, covar_quar = ?
coeffs_50 = np.polyfit(logv,mu,50,w=weight,cov=False) #can't do covariance estimation for this high order

#define a larger range of logv than our data
logv_arr = np.linspace(1,7, 100)

plt.errorbar(logv,mu,yerr=sigma_mu,fmt='b.')
plt.plot(logv_arr, np.polyval(coeffs_lin, logv_arr), label='Linear')
plt.plot(logv_arr, np.polyval(coeffs_quad, logv_arr), label='Quadratic')
plt.plot(logv_arr, np.polyval(coeffs_cub, logv_arr), label='Cubic')
plt.plot(logv_arr, np.polyval(coeffs_quar, logv_arr), label='Quartic')
plt.plot(logv_arr, np.polyval(coeffs_50, logv_arr), label='50th Order')
plt.ylim(25,48)
plt.legend()
plt.show()

As you can see, all the models do a good job of fitting the data in the range of interest, but the way they extrapolate has serious differences. As a result, we need to be extremely cautious when we extend our models past the range of our data. If we don't have a physically motivated model, extrapolation is very dangerous, and even if we do, we still need to be careful. The best practice in this case would be to get more data at higher redshift and then fit again.

That being said, we can make some statements about whether or not we're overfitting using only the data in our range of interest. For that, we turn to measures like the Akaike Information Criterion or the Bayesian Information Criterion:

$BIC = -2~\ln{L_{max}} + k\ln{n} = \chi^2 + k \ln{n}$

$AIC = -2~\ln{L_{max}} + 2 k = \chi^2 +2 k$

In both cases, k is the number of free parameters in the model and for BIC, n is the number of data points that we have. Let's evaluate both BIC and AIC for our linear and quadratic models. AIC and BIC are improved if they <b>decrease</b> significantly. If they increase as we add more parameters, we are absolutely overfitting.

In [ ]:
k_lin = 2
k_quad = 3

bic_linear = chi2_linear+k_lin*np.log(len(logv))
bic_quad = chi2_quad+k_quad*np.log(len(logv))

print('BIC: ')
print(bic_linear, bic_quad)

aic_linear = chi2_linear+2*k_lin
aic_quad = chi2_quad+2*k_quad
print('AIC: ')
print(aic_linear, aic_quad)

In both cases, we increase our information criteria by adding more parameters, and as such, we should treat our linear fit as the best one. For the differences between AIC and BIC and advice about when to use which, bug Jeff Newman.

Non-Linear Regression with Curve Fit
==============

Not sure if we'll have time to go over this in detail, but we are absolutely not restricted to fitting polynomial models to our data, and that is rarely actually the most interesting thing to do. A lot of the time in astronomy, we're interested in emission features, which can be modeled reasonably well with Gaussians. In my research, I think a lot about the [OIII] doublet, which features lines that emit at 5007 and  4959 angstroms with a theoretical 3:1 flux ratio.

Below, we're going to fit a Gaussian to some data from an SDSS galaxy where I have subtracted out the contribution from stars, leaving behind just the emission feature. The wavelength and flux are contained in emline.txt. The model of the gaussian we will be fitting to the doublet will take the following form:

$f(\lambda) = a~(e^\frac{-(\lambda-\mu)^2}{\sigma^2}+\frac{1}{3}~e^\frac{-(\lambda-\mu+48)^2}{\sigma^2})$

a is the absolute normalization, $\mu$ is the median of the stronger line in the doublet, and sigma is the dispersion of the gaussian. The second line is fixed to be 1/3 the peak of the first and to be 48 angstroms blueward.

In the below cell, let's read in the emission line data and plot it.

In [ ]:
emline = np.genfromtxt('emline.txt')
wl = emline[:,0]
flux = emline[:,1]

plt.plot(wl, flux)
plt.xlabel('Wavelength (Angstroms)')
plt.ylabel('Flux')
plt.show()

There's noise, but we can definitely wee two strong peaks centered at the theoretical wavelenghts. Next, let's fit our model to the data and plot using curve fit. Curve fit takes a function that we want to fit to with an arbitrary number of parameters, as well as x, y, and an initial guess. Unlike linear least squares, nonlinear least squares does not necessarily converge. As such, we need to make sure we give a reasonable set of guesses so that the parameters are in a part of parameter space they can find a solution. Those are given in p0.

In [ ]:
from scipy.optimize import curve_fit

def gaussian_OIII(lam, a, mu, sigma):
    ???
    
params, covar = curve_fit(gaussian_OIII, wl, flux, p0=[1, 5007, 10])
a, mu, sigma = params

plt.plot(wl, flux)
plt.plot(wl, gaussian_OIII(wl, a, mu, sigma))
plt.xlabel('Wavelength (Angstroms)')
plt.ylabel('Flux')
plt.show()

Looks pretty good! However, you might notice that there's a secondary peak that appears in both of the lines. Sometimes, in studying emission lines, we like to throw multiple gaussians onto a line and see if we can identify blueshifted or redshifted components that could be the result of outflows, mergers, or other weird stuff going on with the gas. In the below cell, try fitting a two Gaussian model to the line and see if you can capture that blueshifted peak.

$f(\lambda) = a~(e^\frac{-(\lambda-\mu_1)^2}{\sigma_1^2}+\frac{1}{3}~e^\frac{-(\lambda-\mu_1+48)^2}{\sigma_1^2})
~+~ b~(e^\frac{-(\lambda-\mu_2)^2}{\sigma_2^2}+\frac{1}{3}~e^\frac{-(\lambda-\mu_2+48)^2}{\sigma_2^2})$


In [ ]:
def gaussian_OIII_2comp(lam,a,mu1,sigma1,b,mu2,sigma2):
    ???
    
params, covar = curve_fit(gaussian_OIII_2comp, wl, flux, p0=[1, 5007, 10, 1, 5007, 10])
a, mu1, sigma1, b, mu2, sigma2 = params

plt.plot(wl_line, flux_line)
plt.plot(wl_line, gaussian_OIII_2comp(wl, a, mu1, sigma1, b, mu2, sigma2))
plt.xlabel('Wavelength (Angstroms)')
plt.ylabel('Flux')
plt.show()

As you can see, the ability to define our own model in curve fit opens up a ton of fitting possibilities. The trick is coming up with a way to find our parameters, along with the errors in those parameters. In the next lesson, we'll focus on one method of determining the errors in our measurements: bootstrap resampling.